In [1]:
import pandas as pd
import tensorflow as tf

2024-02-23 06:53:29.634708: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 06:53:29.634816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 06:53:29.780390: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv('/kaggle/input/fake-news-dataset/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
# Get independent features
X = df.drop('label', axis =1)
y = df['label']

In [5]:
# Check whether dataset is balanced or not

y.value_counts()

label
1    10413
0    10387
Name: count, dtype: int64

In [6]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense

In [7]:
voc_size = 5000

In [8]:
messages = X.copy()
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [9]:
messages.reset_index(inplace=True)

In [10]:
import nltk
import spacy
import re
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
## Drop null values

df = df.dropna()

In [12]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

ps = PorterStemmer()
corpus = []

for i in range(0, len(messages)):
    # Check if the title is a string
    if isinstance(messages['title'][i], str):
        review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
        review = review.lower()
        review = review.split()
        
        # list comprehension
        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
    else:
        # Handle NaNs or non-string values as per your requirement
        # For example, you could append an empty string to corpus
        corpus.append('')


In [13]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'excerpt draft script donald trump q ampa black church pastor new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find hom

In [14]:
onehot_repr = [one_hot(words,voc_size)for words  in corpus]
onehot_repr

[[4599, 3736, 4593, 4066, 2931, 2115, 3323, 1998, 336, 1340],
 [648, 4054, 314, 2287, 4215, 4060, 4438],
 [3781, 2008, 3680, 3337],
 [3168, 2898, 781, 3703, 302, 2599],
 [2000, 4215, 311, 4195, 2837, 701, 4215, 3320, 2430, 581],
 [1173,
  638,
  300,
  3565,
  2469,
  1814,
  98,
  1382,
  2438,
  3807,
  2594,
  4052,
  2367,
  907,
  4438],
 [2413, 2413, 3091, 3128, 1192, 1137, 3229, 2447, 2364, 4321, 3217, 4898],
 [3908, 3132, 1849, 2868, 2661, 1755, 1006, 3887, 10, 519, 7],
 [1729, 1051, 1770, 844, 1814, 346, 777, 2795, 4253, 2619, 10, 519, 7],
 [1084, 2987, 316, 3425, 2354, 2185, 1814, 4582, 10, 519, 7],
 [3337, 2337, 1768, 2963, 1056, 978, 1591, 4409, 1814, 3175],
 [1592, 4300, 3356, 604, 2698, 94, 3357, 4006],
 [1668, 2995, 4597, 2705, 1289, 4119, 2664, 4376, 2302, 176, 420],
 [3703, 427, 2931, 978, 1814, 2354],
 [4959, 1905, 3631, 3632, 4364, 2432, 1079, 2947, 467],
 [4885, 1513, 468, 4832, 2846, 857, 3180, 10, 519, 7],
 [320, 2549, 3150, 435, 487, 10, 519, 7],
 [4639, 2772, 36

In [15]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1998  336 1340]
 [   0    0    0 ... 4215 4060 4438]
 [   0    0    0 ... 2008 3680 3337]
 ...
 [   0    0    0 ...   10  519    7]
 [   0    0    0 ...  445 2127 2462]
 [   0    0    0 ... 2606 1245 4255]]


In [16]:
## Creating model

# each and every word presented will be converted into size of 40 -> 40 features
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
# o/p is binary so sigmoid
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [17]:
import numpy as np

X_final = np.array(embedded_docs)
y_final = np.array(y)

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [19]:
## Model training

model.fit(X_train,y_train,validation_data =(X_test,y_test),epochs = 10,batch_size=32)

Epoch 1/10
436/436 [==============================] - 12s 19ms/step - loss: 0.2445 - accuracy: 0.9010 - val_loss: 0.1978 - val_accuracy: 0.9229
Epoch 2/10
436/436 [==============================] - 8s 18ms/step - loss: 0.1256 - accuracy: 0.9521 - val_loss: 0.2180 - val_accuracy: 0.9213
Epoch 3/10
436/436 [==============================] - 8s 18ms/step - loss: 0.0755 - accuracy: 0.9733 - val_loss: 0.2407 - val_accuracy: 0.9219
Epoch 4/10
436/436 [==============================] - 8s 18ms/step - loss: 0.0439 - accuracy: 0.9849 - val_loss: 0.2547 - val_accuracy: 0.9223
Epoch 5/10
436/436 [==============================] - 8s 18ms/step - loss: 0.0237 - accuracy: 0.9920 - val_loss: 0.3670 - val_accuracy: 0.9215
Epoch 6/10
436/436 [==============================] - 8s 18ms/step - loss: 0.0136 - accuracy: 0.9958 - val_loss: 0.4289 - val_accuracy: 0.9151
Epoch 7/10
436/436 [==============================] - 8s 18ms/step - loss: 0.0097 - accuracy: 0.9966 - val_loss: 0.5348 - val_accuracy: 0.899

In [20]:
y_pred = model.predict(X_test)

215/215 [==============================] - 2s 5ms/step


In [21]:
y_pred = np.where(y_pred>=0.5,1,0)

In [22]:
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report


confusion_matrix(y_test,y_pred)


array([[3141,  308],
       [ 284, 3131]])

In [23]:
print(accuracy_score(y_test,y_pred))

0.9137529137529138


In [24]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      3449
           1       0.91      0.92      0.91      3415

    accuracy                           0.91      6864
   macro avg       0.91      0.91      0.91      6864
weighted avg       0.91      0.91      0.91      6864

